# MNIST TEST
MNIST data로 학습시킨 모델로 직접 쓴 손글씨 test 후 정확도 판단

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from torchsummary import summary


## Model

In [2]:
# CNN 아키텍처 구성을 위한 클래스 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16,32, kernel_size=3, stride=1, padding=1)

        # Fully Connected Layer
        self.fc = nn.Linear(32*7*7, 10)
        

    def forward(self, x):
        # Convolutional Layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #flatten
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        

        
        return x

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

## Custom Dataset

In [4]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import glob
import os
from PIL import Image
import numpy as np

import torch.optim as optim

In [5]:
class CustomDataset(Dataset):
    def __init__(self,img_path, transform=None):
        self.transform = transform
        file_list =['*.jpg','*.png']
        for files in file_list:
            self.image_paths = glob.glob(os.path.join(img_path,files))
        
    def __getitem__(self, index):
        #이미지 파일 라벨링
        image_path = self.image_paths[index]
        file_name = image_path.split('\\')[-1]
        label = file_name[-5]
        label = np.array(label).astype(np.float32)

        #gray scale
        image = Image.open(image_path).convert('L')

        if self.transform is not None:
            image = self.transform(image)
            
        return image,label
    
        




    def __len__(self):
        return len(self.image_paths)



## Train/Test


In [6]:
if __name__ == '__main__':

    #transformer
    transforms_dataset = transforms.Compose([
        transforms.Resize((28,28)),
        transforms.ToTensor()
    ])
    #이미지 경로
    img_path = './MNIST_data/'
    #Custondata 생성
    total_dataset = CustomDataset(img_path,transforms_dataset)

    #split data
    test_len = int(len(total_dataset)*0.2)
    train_len = len(total_dataset)-test_len
    train, test = random_split(
        total_dataset,
        [train_len,test_len]
    )
    train_dataset = train.dataset
    test_dataset = test.dataset
    

    #DataLoader
    train_loader = DataLoader(train_dataset,
                              batch_size=64,
                              shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=64)


    #Define model, loss, optim
    criterion = nn.CrossEntropyLoss()   # 다중 클래스 분류 문제
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):

        for images, labels in train_loader:
            optimizer.zero_grad()

            # input_image = image.upsqueeze(dim=0)
            outputs = model(images)
            labels = labels.type(torch.LongTensor)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

    correct=0
    total=0
    #Evaluation
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predict = torch.max(outputs.data, 1)

            total +=labels.size(0)
            correct +=(predict==labels).sum().item()

    accuracy = (correct/total) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    





Epoch [1/10], Loss: 2.3560471534729004
Epoch [2/10], Loss: 2.302626848220825
Epoch [3/10], Loss: 2.307903289794922
Epoch [4/10], Loss: 2.311166286468506
Epoch [5/10], Loss: 2.2939682006835938
Epoch [6/10], Loss: 2.2793705463409424
Epoch [7/10], Loss: 2.2881276607513428
Epoch [8/10], Loss: 2.292863607406616
Epoch [9/10], Loss: 2.2947309017181396
Epoch [10/10], Loss: 2.2854039669036865
Test Accuracy: 11.29%
